In [1]:
import os

In [2]:
%pwd

'/Users/aarish/Documents/TextSummer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/aarish/Documents/TextSummer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name : Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml , create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):


        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])
    
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_namr
        )
        
        return data_transformation

In [8]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install datasets

  Using cached datasets-2.13.1-py3-none-any.whl (486 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 5.4 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 8.1 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached multidict-6.0.4-cp311-cp311-macosx_10_9_x86_64.whl (29 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached yarl-1.9.2-cp311-cp311-macosx_10_9_x86_64.whl (64 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.9 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
  Using cached multipro

In [11]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
        
    def convert_examples_to_features(self , example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True)
            
        return{
            'input_ids' : input_encodings['input_ids'],
            'attention_mask' : input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']
        }
    
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataser_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        
        
        

In [14]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
    
except Exception as e:
    raise e

[2023-07-24 01:44:04,549: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-24 01:44:04,554: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-24 01:44:04,555: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'data_transformation'"